This code generates poems using a Hidden Markov Model (HMM) based on Robert Frost's poems as training data. Let's break down what each part of the code does:

Import Statements: The code begins by importing necessary libraries and modules, including numpy, string, sys, and functions from the future module to ensure compatibility with both Python 2 and Python 3.

Data Structures Initialization: Several dictionaries are initialized to store information about the input text:

initial: Stores the distribution of the first words of phrases.
second_word: Stores the distribution of the second word given the first word.
transitions: Stores the transitions between words in the text.
Punctuation Removal: Depending on the Python version being used (2 or 3), the code selects one of two functions (remove_punctuation_2 or remove_punctuation_3) to remove punctuation from lines of text.

Helper Function add2dict: This function adds values to a dictionary under a given key. If the key is not present, it creates a list for that key.

Processing Input Text: The code reads lines from a file called 'robert_frost.txt', tokenizes them into words, converts the words to lowercase, and removes punctuation. It then processes these words to build the following statistics:

initial: Measures the distribution of the first word in a phrase.
second_word: Measures the distribution of the second word given the first word.
transitions: Measures the transitions between words in the text, including the probability of ending a line ('END').
Normalization: The distributions in initial are normalized by dividing the counts by the total count of initial words.

Function list2pdict: This function takes a list of possibilities and converts it into a dictionary of probabilities.

Converting Lists to Probability Distributions: The code converts lists of possibilities in second_word and transitions into dictionaries of probabilities using the list2pdict function.

Function sample_word: This function samples a word from a probability distribution represented as a dictionary. It uses a random number to select a word based on its probability.

Function generate: This function generates four lines of a poem. For each line, it does the following:

Selects an initial word (w0) using the sample_word function.
Selects a second word (w1) based on the distribution in second_word given w0.
Continues to select words based on the transitions in transitions until an 'END' token is encountered. The selected words are added to the sentence.
The generated poem is printed to the console.

Finally, the generate function is called to generate four lines of a poem.
The code aims to create poems in the style of Robert Frost's poetry by modeling the word sequences and transitions in his works. The generated poems are random but influenced by the statistics learned from the training data.

In [3]:
# https://deeplearningcourses.com/c/unsupervised-machine-learning-hidden-markov-models-in-python
# https://udemy.com/unsupervised-machine-learning-hidden-markov-models-in-python
# http://lazyprogrammer.me
# Model and generate Robert Frost poems.
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future



import numpy as np
import string
import sys


initial = {} # start of a phrase
second_word = {}
transitions = {}

# unfortunately these work different ways
def remove_punctuation_2(s):
    return s.translate(None, string.punctuation)

def remove_punctuation_3(s):
    return s.translate(str.maketrans('','',string.punctuation))

if sys.version.startswith('2'):
    remove_punctuation = remove_punctuation_2
else:
    remove_punctuation = remove_punctuation_3


def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)

for line in open('robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split()

    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            # measure the distribution of the first word
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i-1]
            if i == T - 1:
                # measure probability of ending the line
                add2dict(transitions, (t_1, t), 'END')
            if i == 1:
                # measure distribution of second word
                # given only first word
                add2dict(second_word, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(transitions, (t_2, t_1), t)


# normalize the distributions
initial_total = sum(initial.values())
for t, c in iteritems(initial):
    initial[t] = c / initial_total

def list2pdict(ts):
    # turn each list of possibilities into a dictionary of probabilities
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t, 0.) + 1
    for t, c in iteritems(d):
        d[t] = c / n
    return d

for t_1, ts in iteritems(second_word):
    # replace list with dictionary of probabilities
    second_word[t_1] = list2pdict(ts)

for k, ts in iteritems(transitions):
    transitions[k] = list2pdict(ts)

# generate 4 lines
def sample_word(d):
    # print "d:", d
    p0 = np.random.random()
    # print "p0:", p0
    cumulative = 0
    for t, p in iteritems(d):
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False) # should never get here

def generate():
    for i in range(4):
        sentence =[]

        # initial word
        w0 = sample_word(initial)
        sentence.append(w0)

        # sample second word
        w1 = sample_word(second_word[w0])
        sentence.append(w1)

        # second-order transitions until END
        while True:
            w2 = sample_word(transitions[(w0, w1)])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

generate()

# exercise: make them rhyme!

with no small dignity of mien
in eightyfive
its too run down this is a good home i dont mean just skulls of rogers rangers
and put my foot on one without avail
